# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 17 2023 12:46PM,259498,10,9276631,"Digital Brands, LLC",Released
1,Mar 17 2023 12:39PM,259496,10,HH-40681,Hush Hush,Released
2,Mar 17 2023 12:39PM,259496,10,HH-40683,Hush Hush,Released
3,Mar 17 2023 12:39PM,259496,10,HH-40684,Hush Hush,Released
4,Mar 17 2023 12:39PM,259496,10,HH-40685,Hush Hush,Released
5,Mar 17 2023 12:39PM,259496,10,HH-40686,Hush Hush,Released
6,Mar 17 2023 12:39PM,259496,10,HH-40687,Hush Hush,Released
7,Mar 17 2023 12:27PM,259495,10,MSP219656,Methapharm-G,Released
8,Mar 17 2023 12:27PM,259495,10,MSP219657,Methapharm-G,Released
9,Mar 17 2023 12:27PM,259495,10,MSP219658,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,259493,Released,1
8,259494,Released,1
9,259495,Released,3
10,259496,Released,6
11,259498,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259493,NaN,NaN,1.0
259494,NaN,NaN,1.0
259495,NaN,NaN,3.0
259496,NaN,NaN,6.0
259498,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259471,0.0,0.0,1.0
259473,11.0,4.0,8.0
259478,13.0,9.0,0.0
259489,0.0,0.0,4.0
259493,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259471,0,0,1
259473,11,4,8
259478,13,9,0
259489,0,0,4
259493,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259471,0,0,1
1,259473,11,4,8
2,259478,13,9,0
3,259489,0,0,4
4,259493,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259471,,,1
1,259473,11,4,8
2,259478,13,9,
3,259489,,,4
4,259493,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 17 2023 12:46PM,259498,10,"Digital Brands, LLC"
1,Mar 17 2023 12:39PM,259496,10,Hush Hush
7,Mar 17 2023 12:27PM,259495,10,Methapharm-G
10,Mar 17 2023 12:26PM,259494,22,"NBTY Global, Inc."
11,Mar 17 2023 12:12PM,259493,10,"Uniderm USA, Inc"
12,Mar 17 2023 10:58AM,259489,10,"Digital Brands, LLC"
16,Mar 17 2023 7:49AM,259478,19,"AdvaGen Pharma, Ltd"
38,Mar 16 2023 3:37PM,259473,15,"Alliance Pharma, Inc."
61,Mar 16 2023 3:26PM,259471,19,Eli Lilly and Company


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 17 2023 12:46PM,259498,10,"Digital Brands, LLC",,,1
1,Mar 17 2023 12:39PM,259496,10,Hush Hush,,,6
2,Mar 17 2023 12:27PM,259495,10,Methapharm-G,,,3
3,Mar 17 2023 12:26PM,259494,22,"NBTY Global, Inc.",,,1
4,Mar 17 2023 12:12PM,259493,10,"Uniderm USA, Inc",,,1
5,Mar 17 2023 10:58AM,259489,10,"Digital Brands, LLC",,,4
6,Mar 17 2023 7:49AM,259478,19,"AdvaGen Pharma, Ltd",13,9,
7,Mar 16 2023 3:37PM,259473,15,"Alliance Pharma, Inc.",11,4,8
8,Mar 16 2023 3:26PM,259471,19,Eli Lilly and Company,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 12:46PM,259498,10,"Digital Brands, LLC",1,,
1,Mar 17 2023 12:39PM,259496,10,Hush Hush,6,,
2,Mar 17 2023 12:27PM,259495,10,Methapharm-G,3,,
3,Mar 17 2023 12:26PM,259494,22,"NBTY Global, Inc.",1,,
4,Mar 17 2023 12:12PM,259493,10,"Uniderm USA, Inc",1,,
5,Mar 17 2023 10:58AM,259489,10,"Digital Brands, LLC",4,,
6,Mar 17 2023 7:49AM,259478,19,"AdvaGen Pharma, Ltd",,9,13
7,Mar 16 2023 3:37PM,259473,15,"Alliance Pharma, Inc.",8,4,11
8,Mar 16 2023 3:26PM,259471,19,Eli Lilly and Company,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 12:46PM,259498,10,"Digital Brands, LLC",1,,
1,Mar 17 2023 12:39PM,259496,10,Hush Hush,6,,
2,Mar 17 2023 12:27PM,259495,10,Methapharm-G,3,,
3,Mar 17 2023 12:26PM,259494,22,"NBTY Global, Inc.",1,,
4,Mar 17 2023 12:12PM,259493,10,"Uniderm USA, Inc",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 12:46PM,259498,10,"Digital Brands, LLC",1.0,NaN,NaN
1,Mar 17 2023 12:39PM,259496,10,Hush Hush,6.0,NaN,NaN
2,Mar 17 2023 12:27PM,259495,10,Methapharm-G,3.0,NaN,NaN
3,Mar 17 2023 12:26PM,259494,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Mar 17 2023 12:12PM,259493,10,"Uniderm USA, Inc",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 12:46PM,259498,10,"Digital Brands, LLC",1.0,0.0,0.0
1,Mar 17 2023 12:39PM,259496,10,Hush Hush,6.0,0.0,0.0
2,Mar 17 2023 12:27PM,259495,10,Methapharm-G,3.0,0.0,0.0
3,Mar 17 2023 12:26PM,259494,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Mar 17 2023 12:12PM,259493,10,"Uniderm USA, Inc",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1297471,15.0,0.0,0.0
15,259473,8.0,4.0,11.0
19,518949,1.0,9.0,13.0
22,259494,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1297471,15.0,0.0,0.0
1,15,259473,8.0,4.0,11.0
2,19,518949,1.0,9.0,13.0
3,22,259494,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,15.0,0.0,0.0
1,15,8.0,4.0,11.0
2,19,1.0,9.0,13.0
3,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,15.0
1,15,Released,8.0
2,19,Released,1.0
3,22,Released,1.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,22
Status,,,,
Completed,0.0,11.0,13.0,0.0
Executing,0.0,4.0,9.0,0.0
Released,15.0,8.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,22
0,Completed,0.0,11.0,13.0,0.0
1,Executing,0.0,4.0,9.0,0.0
2,Released,15.0,8.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,22
0,Completed,0.0,11.0,13.0,0.0
1,Executing,0.0,4.0,9.0,0.0
2,Released,15.0,8.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()